Retrieval Augmented Generation (검색 증강 생성)

질문과 함께 그 질문에 연관된 개인 DB에 저장된 데이터를 Context로 함께 사용하여 답변을 생성하는 방식
- Model이 가지는 정보가 질문, 모델이 이미 학습한 데이터, Context로 전송된 개인 DB 저장 데이터로 많아짐
- 모델이 이미 학습한 데이터는 참고하지 않고 Context만 참고하여 답변하도록 구성할 수도 있음 (예를 들어, 이미 학습한 데이터에 들어있는 정보가 너무 오래된 데이터일 때 사용)
- 몇 가지 방식이 존재함
    - Stuff 방식
    - Refine 방식 등

---

### Retrieval?
- LangChain Module 중 하나

<img src=".\assets\images/Data_Connection.jpg">

- Data Retrieving Sequence: 여러 Source로부터 데이터들을 Load하고, 각각의 Data를 나눠서(Transform) 임베드(Embed)

---
#### Loader

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader

loader = TextLoader("./assets/LoaderTest/LoaderTest.txt")
loader.load()

[Document(page_content="Part 1, Chapter 1\n\n\nPart One\n\n\n1\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\n\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and h

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./assets/LoaderTest/LoaderTest.pdf")
loader.load()

[Document(page_content='Part 1, Chapter 1 \n \n \nPart One \n \n \n1 \nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin \nnuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors \nof Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering \nalong with him. \n \nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large \nfor indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a \nmetre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly \nhandsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of \ntimes it was seldom working, and at present the electric current was cut off during daylight hours. \nIt was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and \nWinston

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader # text file, powerpoints, html, pdfs, images 등을 모두 처리할 수 있음

loader = UnstructuredFileLoader("./assets/LoaderTest/LoaderTest.docx")
loader.load()

[Document(page_content="Part 1, Chapter 1\n\nPart One\n\n1\n\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\n\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had

In [6]:
len(loader.load()) # 를 실행하면 1개로 나오는데, 모든 텍스트를 하나의 문서로 처리하기 때문임, 나눠주는 작업(transform)을 해줘야 함

1

- 많은 조각을 낼 수록 원하는 데이터를 얻기 쉬워짐

---

#### Transform-1: RecursiveCharacterTextSplitter

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter()

# 다음 두 줄로 작성하거나
# docs = loader.load()
# splitter.split_documents(docs)

# 한 줄로 작성할 수 있음
docs = loader.load_and_split(text_splitter=splitter)
docs


[Document(page_content="Part 1, Chapter 1\n\nPart One\n\n1\n\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\n\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had

In [13]:
len(docs) # 를 실행하면 19개로 나눠진 것을 확인할 수 있음

19

##### Split Size 조절

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200 # 최대 200글자 단위로 나누기
)

docs = loader.load_and_split(text_splitter=splitter)
docs

[Document(page_content='Part 1, Chapter 1\n\nPart One\n\n1', metadata={'source': './assets/LoaderTest/LoaderTest.docx'}),
 Document(page_content='It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors', metadata={'source': './assets/LoaderTest/LoaderTest.docx'}),
 Document(page_content='was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of', metadata={'source': './assets/LoaderTest/LoaderTest.docx'}),
 Document(page_content='cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory', metadata={'source': './assets/LoaderTest/LoaderTest.docx'}),
 Document(page_content

이 경우 문장 중간이 잘린다는 문제점이 발생함

##### Overlap

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200
    , chunk_overlap=50 # 각 단락 사이에 50글자의 중복을 두어 문장이 잘리지 않도록 함
)

docs = loader.load_and_split(text_splitter=splitter)
docs

[Document(page_content='Part 1, Chapter 1\n\nPart One\n\n1', metadata={'source': './assets/LoaderTest/LoaderTest.docx'}),
 Document(page_content='It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors', metadata={'source': './assets/LoaderTest/LoaderTest.docx'}),
 Document(page_content='wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.', metadata={'source': './assets/LoaderTest/LoaderTest.docx'}),
 Document(page_content='The hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a', metadata={'source': './assets/LoaderTest/LoaderTest.docx'}),
 Document(page_content='It depicted simply an enormous face, m

#### Transform-2: CharacterTextSplitter

In [16]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator="\n" # 한 줄 공백을 기준으로 문단을 나눠줌
    , chunk_size=600 # 최대 600글자 단위로 나누지만 문단을 나누는 기준은 separator임, 때문에 600 글자보다 큰 문단도 있을 수 있음
    , chunk_overlap=100
)

docs = loader.load_and_split(text_splitter=splitter)
docs

Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1741, which is longer than the specified 600
Created a chunk of size 2001, which is longer than the specified 600
Created a chunk of size 1900, which is longe

[Document(page_content='Part 1, Chapter 1\nPart One\n1\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.', metadata={'source': './assets/LoaderTest/LoaderTest.docx'}),
 Document(page_content='The hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate

In [17]:
len(docs)

91

#### Transform-3: Custom Length Function

- 실제로 LLM에서 사용하는 단위 개념인 토큰은 글자수와는 전혀 다른 단위로 구분되고 있음
- 각각의 단어들을 Token ID로 변환하고 각 Token ID의 구분에 다라 Token 수를 산정함

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    length_function=len # 글자 수를 기준으로 나누면 정확하지 않음 때문에 Ticktokenizer를 사용함
)

In [18]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter.from_tiktoken_encoder( # from_tiktoken_encoder를 사용하면 토큰화된 문장을 기준으로 나누게 됨
    separator="\n"
    , chunk_size=600 # 최대 600 token 단위로 나눠짐
    , chunk_overlap=100 # 각 단락 사이에 100 token의 중복을 두어 문장이 잘리지 않도록 함
)

docs = loader.load_and_split(text_splitter=splitter)
docs

[Document(page_content='Part 1, Chapter 1\nPart One\n1\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had a varic

---

#### Embedding
- 자연어를 컴퓨터가 처리할 수 있는 언어로 바꾸는 작업
- Embedding을 위해서 먼저 Vetorization 개념에 대해서 알아야 함

##### Vectorization
- nD Vector Space에서 특정 단어들을 각 Axis (속성)에 대해서 Scoring 하는 작업
- OpenAI에서는 1000D 이상의 Vector Space에서 Scoring 함

<img src="./assets/images/Vector Example.png" width=1000>

위 과정을 거쳐 각 Vector에 대한 Searching 과정 (단순히 유무 판단이 아니라 실제 Vector 값이 유사한 단어를 찾는 과정)을 수행할 수 있음
- Vector Searching의 예시로 비슷한 영화를 추천하는 알고리즘 등이 있음
- www.turbomaze.github.io/word2vecjson/ 이라는 비슷한 단어를 찾아주는 사이트도 있음


In [4]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

embedder.embed_query("Hi") # OpenAI에서 Hi를 Embedding 하기 위한 Vector 데이터를 확인할 수 있음

[-0.03629460018406378,
 -0.007184663262302321,
 -0.03371515688153162,
 -0.028660489003748433,
 -0.02683663892458552,
 0.03460102723929279,
 -0.012421715775472752,
 -0.007764386917723339,
 0.0019410967294308348,
 -0.002639696151225509,
 0.024739212823664366,
 -0.002437770038811742,
 -0.005784207816316093,
 -0.0029621265640420295,
 0.00670915973114638,
 -0.0030207502966296384,
 0.033871484972453327,
 -0.0015348017617673204,
 0.021078487187073648,
 -0.008949888444724344,
 -0.021755916364982045,
 0.010337317015461743,
 0.006249940376128115,
 0.007034846979190435,
 -0.01223933114008551,
 0.000850858273119548,
 0.005891684310147391,
 -0.009874840825215944,
 -0.0030451767936668112,
 -0.02464801957464812,
 0.010806306876162615,
 -0.013796117005300009,
 -0.024491689621081163,
 -0.014121804253344064,
 0.002423114047457176,
 -0.018902895178046393,
 0.000547561883680127,
 -0.011255755725498272,
 0.018173354773852176,
 -0.010005116096962616,
 0.01306006293065072,
 -0.01131437875959391,
 -0.00914530

In [3]:
print(len(embedder.embed_query("Hi"))) # 1536 Dimension Vector로 평가하는 것을 확인할 수 있음

1536


In [8]:
vector = embedder.embed_documents([
    "hi"
    , "how"
    , "are"
    , "you"
    , "Longer Example Sentence"
])

print(vector)

print(len(vector))

print(len(vector[0]))

[[-0.030922975329841586, -0.020351571921888304, -0.019417656547076678, -0.0417148842580591, -0.024891437913301454, 0.024372596245366678, -0.0179778709651238, -0.01766656521930488, -0.006504979903020137, -0.015876561371797642, 0.025903180190229098, -0.006991394315954955, -0.017925986984594838, -0.011842565587525743, 0.011414521351177938, 0.016512142461583872, 0.038861256015740406, 0.0005265434145732721, 0.032142251666239935, -0.008710057457404222, -0.019767874812631037, -0.004851171271570787, -0.009410493988512942, -0.01429409344640626, -0.022906867741826747, 0.002458013356446639, 0.010033104548828217, -0.011803652136467734, 0.0025601603505666605, -0.02601991961208055, 0.014475688309580204, 0.0007478619212358725, -0.0357222630865447, -0.01492967453619249, -0.009462378900364478, -0.02470984305012751, 0.006952480864896946, -0.02112983535511304, 0.019145265183638335, -0.005674832954927724, 0.0061936746228625, -0.0007036792798917797, 0.001390334099568663, -0.014488658839051158, -0.023010635

- 매번 모든 문서를 Embedding하는 것은 Token을 매우 많이 소모하기 때문에 최초 Embedding을 시도하면서 해당 내용을 저장(Caching)하는 것이 필요함

---
#### (Vector) Store
- Caching 된 Vector에 대한 Searching이 가능하게 만들어주는 저장 공간
- DB는 유료 또는 무료 형태로 존재함
    - Chroma라는 Local DB 사용 예정
    - 일반적으로 pinecone 등의 DB를 많이 사용
    

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings # text-embedding-ada-002 model을 사용하는 embedder
from langchain.vectorstores import Chroma

# 문서 로드
loader = UnstructuredFileLoader("./assets/LoaderTest/LoaderTest.docx")

# 문서를 CharacterTextSplitter로 나누기
splitter = CharacterTextSplitter(
    separator="\n"
    , chunk_size=600
    , chunk_overlap=100
)
docs = loader.load_and_split(text_splitter=splitter) 

# OpenAIEmbeddings를 사용하여 Embedding 하기
embedder = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs, embedder)

Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1741, which is longer than the specified 600
Created a chunk of size 2001, which is longer than the specified 600
Created a chunk of size 1900, which is longe

In [14]:
# 위 코드에서 저장된 Vectorstore 내부에서 데이터를 검색할 수 있음
results = vectorstore.similarity_search("where does Winston live?")
print(len(results)) # Query와 관련된 4 개의 문서를 반환

results


4


[Document(page_content='There were people sitting all over the stone-flagged floor, and other people, packed tightly together, were sitting on metal bunks, one above the other. Winston and his mother and father found themselves a place on the floor, and near them an old man and an old woman were sitting side by side on a bunk. The old man had on a decent dark suit and a black cloth cap pushed back from very white hair: his face was scarlet and his eyes were blue and full of tears. He reeked of gin. It seemed to breathe out of his skin in place of sweat, and one could have fancied that the tears welling from his eyes were pure gin. But though slightly drunk he was also suffering under some grief that was genuine and unbearable. In his childish way Winston grasped that some terrible thing, something that was beyond forgiveness and could never be remedied, had just happened. It also seemed to him that he knew what it was. Someone whom the old man loved -- a little granddaughter, perhaps h

In [21]:
# Embedding 한 내용을 저장하기 위한 Local 저장소
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")

cache_embedder = CacheBackedEmbeddings.from_bytes_store( 
    embedder
    , cache_dir
)

vectorstore = Chroma.from_documents(docs, cache_embedder) # 최초 caching 시에만 시간이 소요되고, 그 이후에는 빠르게 처리됨

In [24]:
results = vectorstore.similarity_search("where does Winston live?")

results

[Document(page_content='There were people sitting all over the stone-flagged floor, and other people, packed tightly together, were sitting on metal bunks, one above the other. Winston and his mother and father found themselves a place on the floor, and near them an old man and an old woman were sitting side by side on a bunk. The old man had on a decent dark suit and a black cloth cap pushed back from very white hair: his face was scarlet and his eyes were blue and full of tears. He reeked of gin. It seemed to breathe out of his skin in place of sweat, and one could have fancied that the tears welling from his eyes were pure gin. But though slightly drunk he was also suffering under some grief that was genuine and unbearable. In his childish way Winston grasped that some terrible thing, something that was beyond forgiveness and could never be remedied, had just happened. It also seemed to him that he knew what it was. Someone whom the old man loved -- a little granddaughter, perhaps h

---
번외. LangSmith
- 구동하고 있는 LangChain이 어떤 작업을 하고 있는지 확인할 수 있음

---
#### RetrievalQA
- LangChain에서 제공하는 Class 또는 Interface의 일종
- Document 내에서 자료를 검색하여 찾아오는(Retrieve) 기능을 가지고 있음
- OpenAI에서 이미 만들어놓은 몇 가지 Langchain Model

##### 1. Stuff Chain Method
- Query 및 Query와 연관된 모든 Document를 Prompt로서 전달하는 방식

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings # text-embedding-ada-002 model을 사용하는 embedder
from langchain.vectorstores import Chroma
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

chain = RetrievalQA.from_chain_type(
    llm=llm # OpenAI의 기본 모델을 사용
    , chain_type="stuff" # Chain이 답변을 반환하는 방식을 손쉽게 바꿀 수 있음, stuff, refine, ... 등이 있음
    , retriever=vectorstore.as_retriever() # vectorstore를 retriever로 사용, 전환하기 위해 as_retriever()를 사용함

)

chain.run("Where does Winston live?")

'Winston lives in London, which is the chief city of Airstrip One, one of the provinces of Oceania.'

In [26]:
chain.run("What is London?") # London에 대한 일반정보가 아니라, 문서에서 London에 대한 정보를 찾아서 반환함

'London is described as the chief city of Airstrip One, which is itself the third most populous of the provinces of Oceania in the text provided.'

##### 2. Refine Chain Method
- Query를 모든 Document에 Prompt로 전달하고 답변을 얻는 과정을 반복하여 정제(Refine)하는 방식
- Prompt를 전달하는 방식을 반복하기 때문에 더 많은 Token이 소모됨

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS # VectorStore를 바꿔주면 성능에 영향을 미칠 때가 있음
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

vectorstore = FAISS.from_documents(docs, cache_embedder)

llm = ChatOpenAI()

chain = RetrievalQA.from_chain_type(
    llm=llm
    , chain_type="refine"
    , retriever=vectorstore.as_retriever()
)

chain.run("Where does Winston live?")

'Based on the additional context provided, Winston Smith lives in Victory Mansions, a run-down apartment building in London, where he resides on the seventh floor. The building is poorly maintained, and the atmosphere is oppressive, with a constant reminder of surveillance and control through the posters of Big Brother that adorn the walls.'

##### 3. Map Reduced Method
- 각 Document들을 전달 받고 요약을 실행, 해당 요약본을 LLM에 전달하고 LLM에서 답변을 얻어냄

In [30]:
chain = RetrievalQA.from_chain_type(
    llm=llm
    , chain_type="map_reduce"
    , retriever=vectorstore.as_retriever()
)

chain.run("Where does Winston live?")

'Winston lives in Victory Mansions, in a flat that is seven flights up in a building in London.'

##### 4. Map Rerank Method
- Query와 연관된 Document 각각에 대해서 답변을 생성하고, 각 답변에 점수를 매김. 그 중 가장 점수가 높은 답변을 점수와 함께 반환

In [32]:
chain = RetrievalQA.from_chain_type(
    llm=llm
    , chain_type="map_rerank"
    , retriever=vectorstore.as_retriever()
)

chain.run("Where does Winston live?")

c:\Users\Public\Downloads\FullStackGPT\.venv\Lib\site-packages\langchain\chains\llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'Airstrip One, which is a province of Oceania, in the city of London'

- document 기반의 GPT를 구현함
    - Load
    - Spliit
    - Embed
    - Cache
    - Put in VectorStore
    - Run RetrievalQA Chain
        - 일부 Langchain에서 미리 만들어 놓은 (off-the-shelf) 몇 가지 종류의 Chain을 확인함
        - off-the-shelf chain에 대해 알려진 공식적인 문서가 없어 모호하며, Prompt Custom을 위한 정보가 공개되어 있지 않음